# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([ 2, 73, 18, 21, 48, 81, 63, 42, 15,  6,  6,  6,  5, 18, 21, 21, 43,
       42, 75, 18,  1, 67, 53, 67, 81, 70, 42, 18, 63, 81, 42, 18, 53, 53,
       42, 18, 53, 67, 37, 81, 52, 42, 81, 47, 81, 63, 43, 42, 60, 49, 73,
       18, 21, 21, 43, 42, 75, 18,  1, 67, 53, 43, 42, 67, 70, 42, 60, 49,
       73, 18, 21, 21, 43, 42, 67, 49, 42, 67, 48, 70, 42, 36, 62, 49,  6,
       62, 18, 43, 14,  6,  6, 30, 47, 81, 63, 43, 48, 73, 67, 49])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [12]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [13]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [153]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''    
    ## TODO: Get the number of batches we can make
    single_batch_length = batch_size * seq_length
    n_batches = len(arr) // single_batch_length
    total_length = len(arr) - (len(arr) % single_batch_length)
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:total_length]
    
    ## TODO: Reshape into batch_size rows
    # This will create batch_size rows, each row will be total_length / batch_size in length
    # and each row will include (total_length / batch_size) / seq_length sequences, each of length seq_length.
    # You can change the shape of this arr if you want, but that will mean you will need to construct batches differently
    # in the subsequent for loop.
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
#         x = arr[:, n: n + seq_length]
#         # The targets, shifted by one
#         y = arr[:, n + 1: n + 1 + seq_length]
#         yield x, y

        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [154]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [155]:
# printing out the first 10 items in a sequence
# Should produce 8 rows and the first 10 items of the first 50-long sequence as columns.
print('x\n', x[:10, :10])
# x shifted by 1
print('\ny\n', y[:10, :10])

x
 [[ 2 73 18 21 48 81 63 42 15  6]
 [70 36 49 42 48 73 18 48 42 18]
 [81 49 55 42 36 63 42 18 42 75]
 [70 42 48 73 81 42 82 73 67 81]
 [42 70 18 62 42 73 81 63 42 48]
 [82 60 70 70 67 36 49 42 18 49]
 [42 31 49 49 18 42 73 18 55 42]
 [28 65 53 36 49 70 37 43 14 42]]

y
 [[73 18 21 48 81 63 42 15  6  6]
 [36 49 42 48 73 18 48 42 18 48]
 [49 55 42 36 63 42 18 42 75 36]
 [42 48 73 81 42 82 73 67 81 75]
 [70 18 62 42 73 81 63 42 48 81]
 [60 70 70 67 36 49 42 18 49 55]
 [31 49 49 18 42 73 18 55 42 70]
 [65 53 36 49 70 37 43 14 42 79]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [156]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [157]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(input_size=len(tokens), hidden_size=n_hidden, num_layers=n_layers,
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(tokens))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [158]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [159]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=128
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [160]:
batch_size = 5
seq_length = 10
n_epochs = 2 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/2... Step: 10... Loss: 4.2203... Val Loss: 4.1860
Epoch: 1/2... Step: 20... Loss: 3.7499... Val Loss: 3.2993
Epoch: 1/2... Step: 30... Loss: 2.9700... Val Loss: 3.1941
Epoch: 1/2... Step: 40... Loss: 3.1026... Val Loss: 3.1604
Epoch: 1/2... Step: 50... Loss: 3.3272... Val Loss: 3.1542
Epoch: 1/2... Step: 60... Loss: 3.0698... Val Loss: 3.1527
Epoch: 1/2... Step: 70... Loss: 3.1516... Val Loss: 3.1493
Epoch: 1/2... Step: 80... Loss: 3.4242... Val Loss: 3.1498
Epoch: 1/2... Step: 90... Loss: 3.1439... Val Loss: 3.1415
Epoch: 1/2... Step: 100... Loss: 2.9996... Val Loss: 3.1431
Epoch: 1/2... Step: 110... Loss: 3.2460... Val Loss: 3.1371
Epoch: 1/2... Step: 120... Loss: 3.1153... Val Loss: 3.1371
Epoch: 1/2... Step: 130... Loss: 3.2650... Val Loss: 3.1382
Epoch: 1/2... Step: 140... Loss: 3.1323... Val Loss: 3.1366
Epoch: 1/2... Step: 150... Loss: 3.0706... Val Loss: 3.1361
Epoch: 1/2... Step: 160... Loss: 2.9849... Val Loss: 3.1383
Epoch: 1/2... Step: 170... Loss: 3.1636... Val Lo

Epoch: 1/2... Step: 1380... Loss: 2.5197... Val Loss: 2.4947
Epoch: 1/2... Step: 1390... Loss: 2.5407... Val Loss: 2.4836
Epoch: 1/2... Step: 1400... Loss: 2.4453... Val Loss: 2.4845
Epoch: 1/2... Step: 1410... Loss: 2.2981... Val Loss: 2.4859
Epoch: 1/2... Step: 1420... Loss: 2.6600... Val Loss: 2.4804
Epoch: 1/2... Step: 1430... Loss: 2.5009... Val Loss: 2.4721
Epoch: 1/2... Step: 1440... Loss: 2.4470... Val Loss: 2.4776
Epoch: 1/2... Step: 1450... Loss: 2.5882... Val Loss: 2.4760
Epoch: 1/2... Step: 1460... Loss: 2.4416... Val Loss: 2.4706
Epoch: 1/2... Step: 1470... Loss: 2.6153... Val Loss: 2.4614
Epoch: 1/2... Step: 1480... Loss: 2.2979... Val Loss: 2.4683
Epoch: 1/2... Step: 1490... Loss: 2.5825... Val Loss: 2.4635
Epoch: 1/2... Step: 1500... Loss: 2.2846... Val Loss: 2.4632
Epoch: 1/2... Step: 1510... Loss: 2.5546... Val Loss: 2.4573
Epoch: 1/2... Step: 1520... Loss: 2.5770... Val Loss: 2.4748
Epoch: 1/2... Step: 1530... Loss: 2.6993... Val Loss: 2.4487
Epoch: 1/2... Step: 1540

Epoch: 1/2... Step: 2730... Loss: 2.4063... Val Loss: 2.2834
Epoch: 1/2... Step: 2740... Loss: 2.3776... Val Loss: 2.2824
Epoch: 1/2... Step: 2750... Loss: 2.2476... Val Loss: 2.2805
Epoch: 1/2... Step: 2760... Loss: 2.4903... Val Loss: 2.2872
Epoch: 1/2... Step: 2770... Loss: 2.8871... Val Loss: 2.2845
Epoch: 1/2... Step: 2780... Loss: 2.4752... Val Loss: 2.2838
Epoch: 1/2... Step: 2790... Loss: 2.2670... Val Loss: 2.2961
Epoch: 1/2... Step: 2800... Loss: 2.5134... Val Loss: 2.2844
Epoch: 1/2... Step: 2810... Loss: 2.0387... Val Loss: 2.2775
Epoch: 1/2... Step: 2820... Loss: 2.5155... Val Loss: 2.2794
Epoch: 1/2... Step: 2830... Loss: 2.0192... Val Loss: 2.2742
Epoch: 1/2... Step: 2840... Loss: 2.2036... Val Loss: 2.2724
Epoch: 1/2... Step: 2850... Loss: 2.2813... Val Loss: 2.2689
Epoch: 1/2... Step: 2860... Loss: 2.5052... Val Loss: 2.2724
Epoch: 1/2... Step: 2870... Loss: 2.2835... Val Loss: 2.2783
Epoch: 1/2... Step: 2880... Loss: 2.3503... Val Loss: 2.2763
Epoch: 1/2... Step: 2890

Epoch: 1/2... Step: 4080... Loss: 2.6574... Val Loss: 2.1795
Epoch: 1/2... Step: 4090... Loss: 2.2252... Val Loss: 2.1891
Epoch: 1/2... Step: 4100... Loss: 1.9132... Val Loss: 2.1693
Epoch: 1/2... Step: 4110... Loss: 2.4138... Val Loss: 2.1720
Epoch: 1/2... Step: 4120... Loss: 2.0319... Val Loss: 2.1702
Epoch: 1/2... Step: 4130... Loss: 2.2766... Val Loss: 2.1696
Epoch: 1/2... Step: 4140... Loss: 2.8690... Val Loss: 2.1749
Epoch: 1/2... Step: 4150... Loss: 2.1684... Val Loss: 2.1690
Epoch: 1/2... Step: 4160... Loss: 1.9985... Val Loss: 2.1736
Epoch: 1/2... Step: 4170... Loss: 2.0290... Val Loss: 2.1775
Epoch: 1/2... Step: 4180... Loss: 2.0660... Val Loss: 2.1714
Epoch: 1/2... Step: 4190... Loss: 1.8941... Val Loss: 2.1663
Epoch: 1/2... Step: 4200... Loss: 2.4183... Val Loss: 2.1647
Epoch: 1/2... Step: 4210... Loss: 2.2908... Val Loss: 2.1578
Epoch: 1/2... Step: 4220... Loss: 2.4806... Val Loss: 2.1579
Epoch: 1/2... Step: 4230... Loss: 2.0418... Val Loss: 2.1541
Epoch: 1/2... Step: 4240

Epoch: 1/2... Step: 5430... Loss: 2.2149... Val Loss: 2.0926
Epoch: 1/2... Step: 5440... Loss: 2.1862... Val Loss: 2.0914
Epoch: 1/2... Step: 5450... Loss: 2.0719... Val Loss: 2.0843
Epoch: 1/2... Step: 5460... Loss: 1.9238... Val Loss: 2.0788
Epoch: 1/2... Step: 5470... Loss: 2.1348... Val Loss: 2.0827
Epoch: 1/2... Step: 5480... Loss: 1.8173... Val Loss: 2.0782
Epoch: 1/2... Step: 5490... Loss: 2.1514... Val Loss: 2.0779
Epoch: 1/2... Step: 5500... Loss: 2.2329... Val Loss: 2.0804
Epoch: 1/2... Step: 5510... Loss: 1.9233... Val Loss: 2.0819
Epoch: 1/2... Step: 5520... Loss: 2.1604... Val Loss: 2.0804
Epoch: 1/2... Step: 5530... Loss: 2.1498... Val Loss: 2.0850
Epoch: 1/2... Step: 5540... Loss: 1.9622... Val Loss: 2.0810
Epoch: 1/2... Step: 5550... Loss: 1.9141... Val Loss: 2.0838
Epoch: 1/2... Step: 5560... Loss: 1.8447... Val Loss: 2.0825
Epoch: 1/2... Step: 5570... Loss: 2.0424... Val Loss: 2.0821
Epoch: 1/2... Step: 5580... Loss: 1.8469... Val Loss: 2.0810
Epoch: 1/2... Step: 5590

...

Epoch: 2/2... Step: 65340... Loss: 1.5574... Val Loss: 1.6173
Epoch: 2/2... Step: 65350... Loss: 2.0731... Val Loss: 1.6157
Epoch: 2/2... Step: 65360... Loss: 1.6107... Val Loss: 1.6151
Epoch: 2/2... Step: 65370... Loss: 1.7954... Val Loss: 1.6169
Epoch: 2/2... Step: 65380... Loss: 1.7583... Val Loss: 1.6161
Epoch: 2/2... Step: 65390... Loss: 1.7131... Val Loss: 1.6159
Epoch: 2/2... Step: 65400... Loss: 1.1417... Val Loss: 1.6171
Epoch: 2/2... Step: 65410... Loss: 1.8420... Val Loss: 1.6203
Epoch: 2/2... Step: 65420... Loss: 1.5976... Val Loss: 1.6193
Epoch: 2/2... Step: 65430... Loss: 1.4030... Val Loss: 1.6186
Epoch: 2/2... Step: 65440... Loss: 1.5487... Val Loss: 1.6176
Epoch: 2/2... Step: 65450... Loss: 1.2318... Val Loss: 1.6174
Epoch: 2/2... Step: 65460... Loss: 1.3608... Val Loss: 1.6180
Epoch: 2/2... Step: 65470... Loss: 1.6528... Val Loss: 1.6152
Epoch: 2/2... Step: 65480... Loss: 1.4431... Val Loss: 1.6174
Epoch: 2/2... Step: 65490... Loss: 1.6620... Val Loss: 1.6193
Epoch: 2

Epoch: 2/2... Step: 66670... Loss: 1.4237... Val Loss: 1.6132
Epoch: 2/2... Step: 66680... Loss: 1.9386... Val Loss: 1.6146
Epoch: 2/2... Step: 66690... Loss: 1.5249... Val Loss: 1.6149
Epoch: 2/2... Step: 66700... Loss: 1.5856... Val Loss: 1.6142
Epoch: 2/2... Step: 66710... Loss: 1.8937... Val Loss: 1.6129
Epoch: 2/2... Step: 66720... Loss: 2.3357... Val Loss: 1.6117
Epoch: 2/2... Step: 66730... Loss: 1.5534... Val Loss: 1.6125
Epoch: 2/2... Step: 66740... Loss: 1.4937... Val Loss: 1.6141
Epoch: 2/2... Step: 66750... Loss: 1.3124... Val Loss: 1.6146
Epoch: 2/2... Step: 66760... Loss: 1.3180... Val Loss: 1.6137
Epoch: 2/2... Step: 66770... Loss: 1.7404... Val Loss: 1.6126
Epoch: 2/2... Step: 66780... Loss: 1.5402... Val Loss: 1.6133
Epoch: 2/2... Step: 66790... Loss: 1.4547... Val Loss: 1.6134
Epoch: 2/2... Step: 66800... Loss: 1.5505... Val Loss: 1.6129
Epoch: 2/2... Step: 66810... Loss: 1.4761... Val Loss: 1.6131
Epoch: 2/2... Step: 66820... Loss: 1.9234... Val Loss: 1.6144
Epoch: 2

Epoch: 2/2... Step: 68000... Loss: 2.2173... Val Loss: 1.6063
Epoch: 2/2... Step: 68010... Loss: 1.7289... Val Loss: 1.6072
Epoch: 2/2... Step: 68020... Loss: 1.6163... Val Loss: 1.6071
Epoch: 2/2... Step: 68030... Loss: 1.5386... Val Loss: 1.6085
Epoch: 2/2... Step: 68040... Loss: 1.4138... Val Loss: 1.6099
Epoch: 2/2... Step: 68050... Loss: 1.5523... Val Loss: 1.6106
Epoch: 2/2... Step: 68060... Loss: 1.8093... Val Loss: 1.6113
Epoch: 2/2... Step: 68070... Loss: 1.4294... Val Loss: 1.6119
Epoch: 2/2... Step: 68080... Loss: 1.5948... Val Loss: 1.6108
Epoch: 2/2... Step: 68090... Loss: 1.8013... Val Loss: 1.6089
Epoch: 2/2... Step: 68100... Loss: 2.0321... Val Loss: 1.6094
Epoch: 2/2... Step: 68110... Loss: 1.6896... Val Loss: 1.6111
Epoch: 2/2... Step: 68120... Loss: 1.6897... Val Loss: 1.6115
Epoch: 2/2... Step: 68130... Loss: 1.7393... Val Loss: 1.6122
Epoch: 2/2... Step: 68140... Loss: 1.9218... Val Loss: 1.6124
Epoch: 2/2... Step: 68150... Loss: 1.5346... Val Loss: 1.6125
Epoch: 2

Epoch: 2/2... Step: 69330... Loss: 1.9810... Val Loss: 1.6127
Epoch: 2/2... Step: 69340... Loss: 1.5942... Val Loss: 1.6091
Epoch: 2/2... Step: 69350... Loss: 0.9921... Val Loss: 1.6067
Epoch: 2/2... Step: 69360... Loss: 1.7910... Val Loss: 1.6065
Epoch: 2/2... Step: 69370... Loss: 1.5096... Val Loss: 1.6090
Epoch: 2/2... Step: 69380... Loss: 1.4734... Val Loss: 1.6114
Epoch: 2/2... Step: 69390... Loss: 1.6577... Val Loss: 1.6114
Epoch: 2/2... Step: 69400... Loss: 1.7670... Val Loss: 1.6130
Epoch: 2/2... Step: 69410... Loss: 1.2264... Val Loss: 1.6109
Epoch: 2/2... Step: 69420... Loss: 1.5825... Val Loss: 1.6084
Epoch: 2/2... Step: 69430... Loss: 1.4385... Val Loss: 1.6073
Epoch: 2/2... Step: 69440... Loss: 1.1475... Val Loss: 1.6064
Epoch: 2/2... Step: 69450... Loss: 1.6039... Val Loss: 1.6072
Epoch: 2/2... Step: 69460... Loss: 1.6837... Val Loss: 1.6068
Epoch: 2/2... Step: 69470... Loss: 2.2292... Val Loss: 1.6048
Epoch: 2/2... Step: 69480... Loss: 1.9781... Val Loss: 1.6024
Epoch: 2

Epoch: 2/2... Step: 70660... Loss: 1.3725... Val Loss: 1.5980
Epoch: 2/2... Step: 70670... Loss: 1.5075... Val Loss: 1.5966
Epoch: 2/2... Step: 70680... Loss: 1.2475... Val Loss: 1.5972
Epoch: 2/2... Step: 70690... Loss: 1.5613... Val Loss: 1.5976
Epoch: 2/2... Step: 70700... Loss: 1.3995... Val Loss: 1.5981
Epoch: 2/2... Step: 70710... Loss: 2.2824... Val Loss: 1.5981
Epoch: 2/2... Step: 70720... Loss: 1.4317... Val Loss: 1.5989
Epoch: 2/2... Step: 70730... Loss: 1.5497... Val Loss: 1.5996
Epoch: 2/2... Step: 70740... Loss: 1.5057... Val Loss: 1.5999
Epoch: 2/2... Step: 70750... Loss: 2.3400... Val Loss: 1.6004
Epoch: 2/2... Step: 70760... Loss: 2.0130... Val Loss: 1.6017
Epoch: 2/2... Step: 70770... Loss: 1.4667... Val Loss: 1.6021
Epoch: 2/2... Step: 70780... Loss: 1.5041... Val Loss: 1.6021
Epoch: 2/2... Step: 70790... Loss: 1.7912... Val Loss: 1.6020
Epoch: 2/2... Step: 70800... Loss: 1.5410... Val Loss: 1.6005
Epoch: 2/2... Step: 70810... Loss: 1.6802... Val Loss: 1.5981
Epoch: 2

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [161]:
# change the name, for saving multiple files
model_name = 'rnn_2_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [162]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [163]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [164]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna, her.

"Who had seemed the shand of her head to thinks to have his frate that the consecarial of a shooting and the dor of the decilition," said
Alexey Alexandrovitch's, and standed the state, but
his cricked,
which his countess and all her streed the did be staying and strat as the semence, and
any the conscion."

There was all so tree. She considersky that so high the
carriage and word," he said of her
three
step of the master and all and so so thought of her thought
and said.

"What's to her will not her whele that the sone, all without that
how the canders, has seemed in the soul and the possation. And she was to her and with the
more had been tooth, and thing in his condistion. "What's note himself to see it and was no to see the porers.

Stepan Arkadyevitch was not
her to her and
the man anyway of this toon, but that she had nettor in
the sand, that. "If I was
to see that
went all the shoot, and the strild a sersing a tood hand. Stepan Arkadyevitch and a most that he would n

## Loading a checkpoint

In [166]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_2_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [167]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said to
all so shat of
share and at the mithrestioned. There was all a mer and had
sone of
her and with a tinganter word of the sore trates of to she
want, that he said into her time of his stroighten who had sore with her has before if.

"The same whor the same. "I have
to be tomother the child had at the sheep at. "It was as a carriage, but she was a completeness at his sayed as
all
had that thought it was not a compreaches
was a master. And will consereres had to still had hind she saw that to him of the door..." She had have the son, that," said Levin's with she
was to be
shot as he was so hand of the same
hollessifily all his
forgienct hand. "You came annor had she
forgoing the proforsiof he had be shating. Alexey Alexandrovitch, a decined.

"I have some, and thoushed at her had had she feel to the mear to this
sered
to this settling, and this were, he went to her
head and as he say
stroing at the still as imparted that she said. "I am not the marre, when he was the
drac